In [1]:
using DataFrames, ForwardDiff

df = readtable("data/aus/model_australia.txt", separator = ' ', header = false)

,x1,x2,x3,x4
1,1,0,0,0
2,0,1,0,0
3,0,0,1,0
4,35,0,0,0
5,69,34,35,0
6,70,71,70,30
7,0,0,0,1
8,1,0,0,0
9,0,1,0,0
10,0,0,1,0


In [2]:
function f(β::Vector, n::Number = 210)
    i = 1
    model = 0
    while i <= n
        model += f_ind(i, β)
        i += 1
    end
    return model/n
end

function f_ind(i::Number, β::Vector)
    d = 0
    data = convert(Array, df[i*7-3:i*7-1, :])
    choice = convert(Array, df[i*7:i*7, :])
    id = find(choice .== 1)
    alt = find(choice .== 0)
    c = exp(dot(vec(data[:, id]), β))
    for j in 1:length(alt)
        d += exp(dot(vec(data[:, alt[j]]), β))
    end
    return log(c/(c+d))
end

f_ind (generic function with 1 method)

In [3]:
function g(β::Vector, n::Number = 210)
    i = 1
    g = 0
    while i <= n
        g += g_ind(i, β)
        i += 1
    end
    return g
end

function g_ind(i::Number, β::Vector)
    f = f_ind(i, β)
    return ForwardDiff.gradient(f, β)
end

function g!(β::Vector, storage::Vector)
    s = g(β)
    storage[1:length(s)] = s[1:length(s)]
end

g! (generic function with 1 method)

In [4]:
function H(β::Vector, n::Number = 210)
    i = 1
    H = 0
    while i <= n
        H += H_ind(i, β)
        i += 1
    end
    return H
end

function H_ind(i::Number, β::Vector)
    f = f_ind(i, β)
    return ForwardDiff.hessian(f, β)
end

function H!(β::Vector, storage::Matrix)
    s = H(β)
    n, m = size(s)
    storage[1:n, 1:m] = s[1:length(s)]
end

H! (generic function with 1 method)